In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

In [2]:
import pandas as pd
import numpy as np
from bayesian.train_bn import structure_learning, parameter_learning, parameter_learning_mix
from preprocess.discretization import get_nodes_type, discretization, inverse_discretization, code_categories
from bayesian.save_bn import save_structure, save_params, read_structure, read_params
from external.libpgm.hybayesiannetwork import HyBayesianNetwork
from visualization.visualization import draw_BN
from bayesian.calculate_accuracy import calculate_acc
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from copy import copy
from external.libpgm.sampleaggregator import SampleAggregator
import operator
from sklearn.metrics import accuracy_score, mean_squared_error
from scipy import stats
from scipy.stats import multivariate_normal
from sklearn.mixture import GaussianMixture
import math
from pomegranate import DiscreteDistribution
import random
from sklearn.model_selection import train_test_split

e:\BAMT-master\external\libpgm\hybayesiannetwork.py:120: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert ((isinstance(n, int) and n > 0), "Argument must be a positive integer.")
e:\BAMT-master\external\libpgm\hybayesiannetwork.py:143: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (pvalue != 'default', "Graph skeleton was not topologically ordered.")


In [3]:
df_result_accuracy = pd.DataFrame(columns = ['Parameter', 'Strategy', 'Alg', 'accuracy'])
df_result_rmse = pd.DataFrame(columns = ['Parameter', 'Strategy', 'Alg', 'rmse'])
method = 'MI'

In [4]:
geo = pd.read_csv('../data/hackathon_processed.csv')
columns = ['Tectonic regime', 'Period', 'Lithology', 'Structural setting', 'Hydrocarbon type', 'Gross','Netpay','Porosity','Permeability', 'Depth']
"""geo = pd.read_csv('../datasets/healthcare.csv')
geo = geo.drop(columns=['Unnamed: 0'])
columns = ['A', 'C', 'H', 'D', 'I', 'O', 'T']"""
"""geo = pd.read_csv('../datasets/sangiovese.csv')
geo = geo.drop(columns=['Unnamed: 0'])
columns = ['Treatment', 'SproutN', 'BunchN', 'GrapeW', 'WoodW',
       'SPAD06', 'NDVI06', 'SPAD08', 'NDVI08', 'Acid', 'Potass', 'Brix', 'pH',
       'Anthoc', 'Polyph']"""
geo = geo[columns]
geo.dropna(inplace=True)
geo.reset_index(inplace=True, drop=True)

In [5]:
geo_train, geo_test = train_test_split(geo, test_size=0.1)
geo_train.reset_index(inplace=True, drop=True)
geo_test.reset_index(inplace=True, drop=True)

In [6]:
geo_types = get_nodes_type(geo)
geo_types

{'Tectonic regime': 'disc',
 'Period': 'disc',
 'Lithology': 'disc',
 'Structural setting': 'disc',
 'Hydrocarbon type': 'disc',
 'Gross': 'cont',
 'Netpay': 'cont',
 'Porosity': 'cont',
 'Permeability': 'cont',
 'Depth': 'cont'}

In [7]:
colums_for_code = []
columns_for_disc = []
for c in columns:
    if geo_types[c] == 'disc':
        colums_for_code.append(c)
    else:
        columns_for_disc.append(c)

In [8]:
geo_coded, label_coder = code_categories(geo_train, 'label', colums_for_code)
geo_discrete, coder = discretization(geo_coded, 'equal_frequency', columns_for_disc)
geo_only_discrete, discrete_coder = discretization(geo_train, 'equal_frequency', columns_for_disc)

In [29]:
geo_new_types = {'Tectonic regime': 'disc',
 'Period': 'disc',
 'Lithology': 'disc',
 'Structural setting': 'disc',
 'Hydrocarbon type': 'disc',
 'Gross': 'disc',
 'Netpay': 'disc',
 'Porosity': 'disc',
 'Permeability': 'disc',
 'Depth': 'disc'}

# EVO

# Дискретная стурктура + смешанные параметры

In [11]:
bn_geo = structure_learning(geo_discrete, 'evo', geo_types, method)

Best metric is 10.0105
Generation num: 0
max_depth: 8, no improvements: 0
Number of mutation attempts exceeded. Please check composer requirements for correctness.
spent time: 1.5 min
Best metric is 9.999099999999999
Generation num: 1
max_depth: 8, no improvements: 0
Number of mutation attempts exceeded. Please check composer requirements for correctness.
Number of mutation attempts exceeded. Please check composer requirements for correctness.
Number of mutation attempts exceeded. Please check composer requirements for correctness.
Number of mutation attempts exceeded. Please check composer requirements for correctness.
Number of mutation attempts exceeded. Please check composer requirements for correctness.
Number of mutation attempts exceeded. Please check composer requirements for correctness.
Number of mutation attempts exceeded. Please check composer requirements for correctness.
Number of mutation attempts exceeded. Please check composer requirements for correctness.
Number of mu

In [12]:
draw_BN(bn_geo, geo_types, 'geo_net')

In [13]:
print(bn_geo)

{'V': ['Tectonic regime', 'Period', 'Lithology', 'Structural setting', 'Hydrocarbon type', 'Gross', 'Netpay', 'Porosity', 'Permeability', 'Depth'], 'E': [['Period', 'Porosity'], ['Netpay', 'Permeability'], ['Tectonic regime', 'Porosity'], ['Tectonic regime', 'Permeability'], ['Lithology', 'Period'], ['Hydrocarbon type', 'Tectonic regime'], ['Structural setting', 'Period'], ['Structural setting', 'Tectonic regime']]}


In [14]:
params = parameter_learning(geo_train, geo_types, bn_geo, 'simple')

In [15]:
save_structure(bn_geo, 'geo_net')
skel = read_structure('geo_net')
save_params(params, 'geo_net_param')
params = read_params('geo_net_param')
geo_bn = HyBayesianNetwork(skel, params)

In [16]:
acc, rmse, real_data, pred_data = calculate_acc(geo_bn, geo_test, columns, 'nrmse')

"['GLAUCONITIC SANDSTONE', 'RIFT']"


In [17]:
acc

{'Tectonic regime': 0.76,
 'Period': 0.32,
 'Lithology': 0.51,
 'Structural setting': 0.31,
 'Hydrocarbon type': 0.67}

In [18]:
rmse

{'Gross': 0.24,
 'Netpay': 0.19,
 'Porosity': 0.62,
 'Permeability': 0.25,
 'Depth': 0.46}

In [19]:
for key in acc.keys():
    df_result_accuracy = df_result_accuracy.append({'Parameter': key, 'Strategy': 'evo', 'Alg':'D+M','accuracy':acc[key]}, ignore_index=True)
for key in rmse.keys():
    df_result_rmse = df_result_rmse.append({'Parameter': key, 'Strategy': 'evo', 'Alg':'D+M','rmse':rmse[key]}, ignore_index=True)

# Смешанная структура + смешанные параметры

In [20]:
bn_geo = structure_learning(geo_coded, 'evo', geo_types, method)

Best metric is 13.508421804077068
Generation num: 0
max_depth: 8, no improvements: 0
Number of mutation attempts exceeded. Please check composer requirements for correctness.
spent time: 2.3 min
Best metric is 13.508421804077068
Generation num: 1
max_depth: 8, no improvements: 1
spent time: 3.1 min
Best metric is 13.508421804077068
Generation num: 2
max_depth: 8, no improvements: 2
spent time: 3.8 min
Best metric is 13.508421804077068
Generation num: 3
max_depth: 8, no improvements: 3
spent time: 4.7 min
Best metric is 13.508421804077068
Generation num: 4
max_depth: 8, no improvements: 4
spent time: 5.5 min
Best metric is 13.508421804077068
Generation num: 5
max_depth: 8, no improvements: 5
spent time: 6.3 min
Best metric is 13.508421804077068
Generation num: 6
max_depth: 8, no improvements: 6
spent time: 7.1 min
Best metric is 13.508421804077068
Generation num: 7
max_depth: 8, no improvements: 7
spent time: 7.9 min
Best metric is 13.508421804077068
Generation num: 8
max_depth: 8, no i

In [21]:
draw_BN(bn_geo, geo_types, 'geo_net')

In [22]:
params = parameter_learning(geo_train, geo_types, bn_geo, 'simple')

In [23]:
save_structure(bn_geo, 'geo_net')
skel = read_structure('geo_net')
save_params(params, 'geo_net_param')
params = read_params('geo_net_param')
geo_bn = HyBayesianNetwork(skel, params)

In [24]:
acc, rmse, real, pred = calculate_acc(geo_bn, geo_test, columns, 'nrmse')

In [25]:
acc

{'Tectonic regime': 0.58,
 'Period': 0.27,
 'Lithology': 0.51,
 'Structural setting': 0.31,
 'Hydrocarbon type': 0.67}

In [26]:
rmse

{'Gross': 0.24,
 'Netpay': 0.19,
 'Porosity': 0.62,
 'Permeability': 0.25,
 'Depth': 0.46}

In [27]:
for key in acc.keys():
    df_result_accuracy = df_result_accuracy.append({'Parameter': key, 'Strategy': 'evo', 'Alg':'M+M','accuracy':acc[key]}, ignore_index=True)
for key in rmse.keys():
    df_result_rmse = df_result_rmse.append({'Parameter': key, 'Strategy': 'evo', 'Alg':'M+M','rmse':rmse[key]}, ignore_index=True)

# Дискретная структура + дискретные параметры

In [30]:
bn_geo = structure_learning(geo_discrete, 'evo', geo_new_types, method)

Best metric is 9.6414
Generation num: 0
max_depth: 8, no improvements: 0
spent time: 0.2 min
Best metric is 9.6414
Generation num: 1
max_depth: 8, no improvements: 1
spent time: 0.3 min
Best metric is 9.5136
Generation num: 2
max_depth: 8, no improvements: 0
spent time: 0.4 min
Best metric is 9.4265
Generation num: 3
max_depth: 8, no improvements: 0
spent time: 0.7 min
Best metric is 9.4147
Generation num: 4
max_depth: 8, no improvements: 0
spent time: 0.8 min
Best metric is 9.3558
Generation num: 5
max_depth: 8, no improvements: 0
spent time: 0.9 min
Best metric is 9.3443
Generation num: 6
max_depth: 8, no improvements: 0
spent time: 1.0 min
Best metric is 9.3356
Generation num: 7
max_depth: 8, no improvements: 0
spent time: 1.1 min
Best metric is 9.3338
Generation num: 8
max_depth: 8, no improvements: 0
spent time: 1.3 min
Best metric is 9.3245
Generation num: 9
max_depth: 8, no improvements: 0
spent time: 1.4 min
Best metric is 9.3245
Generation num: 10
max_depth: 8, no improvements

In [31]:
draw_BN(bn_geo, geo_types, 'geo_net')

In [32]:
params = parameter_learning(geo_only_discrete, geo_new_types, bn_geo, 'simple')

In [33]:
save_structure(bn_geo, 'geo_net')
skel = read_structure('geo_net')
save_params(params, 'geo_net_param')
params = read_params('geo_net_param')
geo_bn = HyBayesianNetwork(skel, params)

In [34]:
test_discrete = copy(geo_test)
test_discrete[columns_for_disc] = discrete_coder.transform(geo_test[columns_for_disc].values) 

In [35]:
for c in columns_for_disc:
    test_discrete[c] = test_discrete[c].apply(lambda x: int(x))
test_discrete

,Tectonic regime,Period,Lithology,Structural setting,Hydrocarbon type,Gross,Netpay,Porosity,Permeability,Depth
0,GRAVITY,JURASSIC,LIMESTONE,SALT,OIL,1,2,4,3,2
1,EXTENSION,NEOGENE,SANDSTONE,RIFT,OIL,4,1,4,4,1
2,COMPRESSION,CRETACEOUS,SANDSTONE,FORELAND,OIL,0,0,3,1,1
3,COMPRESSION,PALEOGENE-NEOGENE,LIMESTONE,FORELAND,OIL,4,4,3,2,0
4,GRAVITY,CRETACEOUS,SANDSTONE,SALT,OIL,2,1,4,4,1
5,EXTENSION,PALEOGENE,SANDSTONE,RIFT,OIL,1,1,4,4,1
6,COMPRESSION,ORDOVICIAN,SANDSTONE,INTRACRATONIC,GAS,4,3,0,0,2
7,EXTENSION,MESOZOIC,VOLCANICS,RIFT,OIL,3,3,1,0,4
8,STRIKE-SLIP,NEOGENE,SANDSTONE,INVERSION,OIL,4,3,3,2,1
9,COMPRESSION,CRETACEOUS,LIMESTONE,SALT,OIL,2,2,3,2,1


In [36]:
acc, rmse, real, pred = calculate_acc(geo_bn, test_discrete, columns, 'nrmse')

"['OIL', 'MESOZOIC']"
"['4', 'MESOZOIC']"
"['3', '1', 'MESOZOIC']"
"['GAS-CONDENSATE', 'DEVONIAN-PERMIAN']"
"['4', 'DEVONIAN-PERMIAN']"
"['4', '0', 'DEVONIAN-PERMIAN']"
"['OIL', 'PALEOZOIC-CRETACEOUS']"
"['4', 'PALEOZOIC-CRETACEOUS']"
"['4', '2', 'PALEOZOIC-CRETACEOUS']"
"['OIL', 'PALEOZOIC']"
"['4', 'PALEOZOIC']"
"['4', '2', 'PALEOZOIC']"
e:\Graphologists-master\venv\lib\site-packages\sklearn\metrics\_classification.py:208: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score = y_true == y_pred
e:\Graphologists-master\venv\lib\site-packages\sklearn\metrics\_classification.py:208: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score = y_true == y_pred
e:\Graphologists-master\venv\lib\site-packages\sklearn\metrics\_classification.py:208: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future w

In [37]:
acc

{'Tectonic regime': 0.58,
 'Period': 0.27,
 'Lithology': 0.51,
 'Structural setting': 0.31,
 'Hydrocarbon type': 0.67,
 'Gross': 0.0,
 'Netpay': 0.0,
 'Porosity': 0.0,
 'Permeability': 0.0,
 'Depth': 0.0}

In [38]:
for key in colums_for_code:
    df_result_accuracy = df_result_accuracy.append({'Parameter': key, 'Strategy': 'evo', 'Alg':'D+D','accuracy':acc[key]}, ignore_index=True)

In [40]:
invers_real = inverse_discretization(pd.DataFrame(np.transpose(real[5:10]), columns=['Gross', 'Netpay', 'Porosity', 'Permeability', 'Depth']), ['Gross', 'Netpay', 'Porosity', 'Permeability', 'Depth'], discrete_coder)

ValueError: Shape of passed values is (5, 1), indices imply (5, 5)

In [48]:
invers_pred = inverse_discretization(pd.DataFrame(np.transpose(pred[5:10]), columns=['Gross', 'Netpay', 'Porosity', 'Permeability', 'Depth']), ['Gross', 'Netpay', 'Porosity', 'Permeability', 'Depth'], discrete_coder)

In [49]:
mean_squared_error(invers_real['Gross'], invers_pred['Gross'], squared=False)
df_result_rmse = df_result_rmse.append({'Parameter': 'Gross', 'Strategy': 'evo', 'Alg':'D+D','rmse':mean_squared_error(invers_real['Gross'], invers_pred['Gross'], squared=False)}, ignore_index=True)

In [50]:
mean_squared_error(invers_real['Netpay'], invers_pred['Netpay'], squared=False)
df_result_rmse = df_result_rmse.append({'Parameter': 'Netpay', 'Strategy': 'evo', 'Alg':'D+D','rmse':mean_squared_error(invers_real['Netpay'], invers_pred['Netpay'], squared=False)}, ignore_index=True)

In [51]:
mean_squared_error(invers_real['Porosity'], invers_pred['Porosity'], squared=False)
df_result_rmse = df_result_rmse.append({'Parameter': 'Porosity', 'Strategy': 'evo', 'Alg':'D+D','rmse':mean_squared_error(invers_real['Porosity'], invers_pred['Porosity'], squared=False)}, ignore_index=True)


In [52]:
mean_squared_error(invers_real['Permeability'], invers_pred['Permeability'], squared=False)
df_result_rmse = df_result_rmse.append({'Parameter': 'Permeability', 'Strategy': 'evo', 'Alg':'D+D','rmse':mean_squared_error(invers_real['Permeability'], invers_pred['Permeability'], squared=False)}, ignore_index=True)

In [53]:
mean_squared_error(invers_real['Depth'], invers_pred['Depth'], squared=False)
df_result_rmse = df_result_rmse.append({'Parameter': 'Depth', 'Strategy': 'evo', 'Alg':'D+D','rmse':mean_squared_error(invers_real['Depth'], invers_pred['Depth'], squared=False)}, ignore_index=True)

# Смешанная структура + дискретные параметры

In [41]:
bn_geo = structure_learning(geo_coded, 'evo', geo_new_types, method)

Best metric is 19.273121801641018
Generation num: 0
max_depth: 8, no improvements: 0
spent time: 1.7 min
Best metric is 18.56163884911347
Generation num: 1
max_depth: 8, no improvements: 0
spent time: 2.6 min
Best metric is 17.63554015455071
Generation num: 2
max_depth: 8, no improvements: 0
spent time: 3.3 min
Best metric is 17.63554015455071
Generation num: 3
max_depth: 8, no improvements: 1
spent time: 4.1 min
Best metric is 17.63554015455071
Generation num: 4
max_depth: 8, no improvements: 2
Number of mutation attempts exceeded. Please check composer requirements for correctness.
spent time: 6.0 min
Best metric is 17.63554015455071
Generation num: 5
max_depth: 8, no improvements: 3
spent time: 6.8 min
Best metric is 17.63554015455071
Generation num: 6
max_depth: 8, no improvements: 4
spent time: 7.6 min
Best metric is 17.63554015455071
Generation num: 7
max_depth: 8, no improvements: 5
spent time: 8.4 min
Best metric is 17.63554015455071
Generation num: 8
max_depth: 8, no improveme

In [42]:
draw_BN(bn_geo, geo_types, 'geo_net')

In [43]:
params = parameter_learning(geo_only_discrete, geo_new_types, bn_geo, 'simple')

In [44]:
save_structure(bn_geo, 'geo_net')
skel = read_structure('geo_net')
save_params(params, 'geo_net_param')
params = read_params('geo_net_param')
geo_bn = HyBayesianNetwork(skel, params)

In [45]:
acc, rmse, real, pred = calculate_acc(geo_bn, test_discrete, columns, 'nrmse')

"['0', 'MESOZOIC']"
"['1', 'DEVONIAN-PERMIAN']"
"['3', 'PALEOZOIC-CRETACEOUS']"
"['GLAUCONITIC SANDSTONE', '0']"
"['2', 'PALEOZOIC']"
e:\Graphologists-master\venv\lib\site-packages\sklearn\metrics\_classification.py:208: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score = y_true == y_pred
e:\Graphologists-master\venv\lib\site-packages\sklearn\metrics\_classification.py:208: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score = y_true == y_pred
e:\Graphologists-master\venv\lib\site-packages\sklearn\metrics\_classification.py:208: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score = y_true == y_pred
e:\Graphologists-master\venv\lib\site-packages\sklearn\metrics\_classification.py:208: FutureWarning: elementwise comparison failed; returning sca

In [46]:
acc

{'Tectonic regime': 0.58,
 'Period': 0.27,
 'Lithology': 0.51,
 'Structural setting': 0.31,
 'Hydrocarbon type': 0.67,
 'Gross': 0.0,
 'Netpay': 0.0,
 'Porosity': 0.0,
 'Permeability': 0.0,
 'Depth': 0.0}

In [47]:
for key in colums_for_code:
    df_result_accuracy = df_result_accuracy.append({'Parameter': key, 'Strategy': 'evo', 'Alg':'M+D','accuracy':acc[key]}, ignore_index=True)

In [48]:
invers_real = inverse_discretization(pd.DataFrame(np.transpose(real[5:10]), columns=['Gross', 'Netpay', 'Porosity', 'Permeability', 'Depth']), ['Gross', 'Netpay', 'Porosity', 'Permeability', 'Depth'], discrete_coder)


e:\Graphologists-master\venv\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


ValueError: Shape of passed values is (5, 1), indices imply (5, 5)

In [ ]:
invers_pred = inverse_discretization(pd.DataFrame(np.transpose(pred[5:10]), columns=['Gross', 'Netpay', 'Porosity', 'Permeability', 'Depth']), ['Gross', 'Netpay', 'Porosity', 'Permeability', 'Depth'], discrete_coder)

In [ ]:
mean_squared_error(invers_real['Gross'], invers_pred['Gross'], squared=False)
df_result_rmse = df_result_rmse.append({'Parameter': 'Gross', 'Strategy': 'evo', 'Alg':'M+D','rmse':mean_squared_error(invers_real['Gross'], invers_pred['Gross'], squared=False)}, ignore_index=True)

In [ ]:
mean_squared_error(invers_real['Netpay'], invers_pred['Netpay'], squared=False)
df_result_rmse = df_result_rmse.append({'Parameter': 'Netpay', 'Strategy': 'evo', 'Alg':'M+D','rmse':mean_squared_error(invers_real['Netpay'], invers_pred['Netpay'], squared=False)}, ignore_index=True)

In [ ]:
mean_squared_error(invers_real['Porosity'], invers_pred['Porosity'], squared=False)
df_result_rmse = df_result_rmse.append({'Parameter': 'Porosity', 'Strategy': 'evo', 'Alg':'M+D','rmse':mean_squared_error(invers_real['Porosity'], invers_pred['Porosity'], squared=False)}, ignore_index=True)

In [ ]:
mean_squared_error(invers_real['Permeability'], invers_pred['Permeability'], squared=False)
df_result_rmse = df_result_rmse = df_result_rmse.append({'Parameter': 'Permeability', 'Strategy': 'evo', 'Alg':'M+D','rmse':mean_squared_error(invers_real['Permeability'], invers_pred['Permeability'], squared=False)}, ignore_index=True)

In [ ]:
mean_squared_error(invers_real['Depth'], invers_pred['Depth'], squared=False)
df_result_rmse = df_result_rmse.append({'Parameter': 'Depth', 'Strategy': 'evo', 'Alg':'M+D','rmse':mean_squared_error(invers_real['Depth'], invers_pred['Depth'], squared=False)}, ignore_index=True)

In [ ]:
df_result_accuracy.to_csv('geo_results_disc.csv', index=False)
df_result_rmse.to_csv('geo_results_cont.csv', index=False)